### web crawing - requests
- 인터넷에서는 client가 request를 보내고 서버에서 response를 보냄
- 웹크롤링을 위해서 코드에서 서버로 request를 보내고 response를 받게 됨
- JSON : 일반적으로 서버에서 클라이언트로 데이터를 보낼 때 사용하는 양식. 클라이언트가 사용하는 언어에 관계 없이 통일된 데이터를 주고받을 수 있도록, 일정한 패턴을 지닌 문자열을 생성해 내보내면 클라이언트는 그를 해석해 데이터를 자기만의 방식으로 온전히 저장, 표시할 수 있게 됨
---
과거 웹 초기 시절부터 사용되어 온 XML은 헤더와 태그 등의 여러 요소로 가독성이 떨어지고, 쓸데없이 용량을 잡아먹는다는 단점이 항상 지적되어 왔다. 이에 대응해 간결하고 통일된 양식으로 각광을 받고 있는 것이 JSON이다. 

`방법` : 1 - 2 - 3 순서대로 사용 추천
1. API를 통해 JSON 포맷으로 데이터를 받음, 문자열로 request, reponse를 주고 받으며, 문자열로 받은 것을 JSON형태의 obj로 parsing함
    * JSON 데이터만 있으므로 2보다 크롤링 속도가 빠름
2. request를 던진 후 HTML 코드를 문자열로 받아와서 CSS sellecor로 특정 엘리먼트에 있는 것을 가져오도록 parsing함, CSS selector를 obj로 사용(BS4 패키지 이용)
3. selenium은 frontend page 테스팅 툴이었지만 Selenium을 이용해서 데이터를 가져오는데 사용, 직접 browser를 띄워서 데이터를 수집(CSS selector이용)
4. scrapy : 웹크롤링을 위해 만들어진 파이썬 패키지 - 홈페이지+robots.txt 처럼 정책설정 가능, scrapy도 동적인 웹사이트 크롤링하려면 selenium 필요

---
- 정적인 웹페이지: html 코드 보여주기만 하는 홈페이지 - 주로 2번 사용, html 코드를 다 실행 후 데이터 수집
- 동적인 웹페이지: API를 통해 Javascript를 이용해서 서버에서 화면을 계속 바뀌게 보여줌 - 3번 사용
---
서버에는 화면이 없으므로 local코드를 넣으면 오류남, 해결방법으로
1. 크롬 driver headless 이용 : 예전에는 화면을 띄우지 않고 크롤링 지원하는 드라이버인 PhantomJS를 썼지만 더이상 selenium에서 지원 안함
2. Xufb : 서버 메모리 상에 virtual window를 가상으로 띄워주는 툴

---
- api json을 사용 : 네이버 주식 데이터 크롤링
- api json을 사용, forecastio : dark sky api로 날씨 데이터를 수집
- bs4 : 네이버 실시간 키워드, 다음 실시간 키워드 크롤링
- web file url로 파일을 다운로드 받는 방법

In [2]:
import requests

In [3]:
from bs4 import BeautifulSoup

In [4]:
import forecastio


- 홈페이지 더보기 버튼은 API를 통해 서버에 요청해 데이터를 보여줌 - 동적페이지
- 개발자 페이지에서 Network > XHR > (트래픽 발생시킨 후) > 생성된 항목 클릭 > Headers > General > request URL(가져다 쓸 데이터, JSON형태)
    - Request Method가 get방식이면 url주소의 쿼리를 통해 주소 해석
    - Remote Address : 찾아간 아이피
    - Request Headers는 client가 요청할 때 보내는 정보
    - Request Headers > Referer : 이전페이지가 뭐였는지 보여줌

In [5]:
# 첫번째 방법
def make_url(pageSize=10, page=1):
    return "http://m.stock.naver.com/api/json/sise/siseListJson.nhn?\
menu=market_sum&sosok=0&pageSize=" + str(pageSize) + "&page=" + str(page)
    
make_url()

'http://m.stock.naver.com/api/json/sise/siseListJson.nhn?menu=market_sum&sosok=0&pageSize=10&page=1'

In [6]:
#get으로 데이터 요청 post 쓰면 `requests.post`로 데이터 요청, 개발자모드에서 get방식이었기 때문에 get씀
def get_data(url):
    response = requests.get(url)  
    json_info = response.json()   #JSON으로 변환
    companys = json_info['result']['itemList']
    
    df = pd.DataFrame(columns=['종목', '시세', '전일비', '등락율', '시가총액', '거래량'])
    for company in companys:
        df.loc[len(df)] = {
            '종목':company['nm'],
            '시세':company['nv'],
            '전일비':company['cv'],
            '등락율':company['cr'],
            '시가총액':company['mks'],
            '거래량':company['aq'],
        }
    return df

In [7]:
url = make_url()
get_data(url)

,종목,시세,전일비,등락율,시가총액,거래량
0,삼성전자,46600,-1050,-2.20,2991405,16606159
1,SK하이닉스,84000,-3000,-3.45,611522,4710201
2,셀트리온,305000,6500,2.18,382458,1630687
3,삼성전자우,37200,-600,-1.59,336150,860365
4,POSCO,356000,-9000,-2.47,310385,194950
5,현대차,135000,1000,0.75,297373,497478
6,삼성바이오로직스,407500,-500,-0.12,269622,146525
7,LG화학,366500,-7000,-1.87,258721,263375
8,KB금융,55500,500,0.91,232052,788519
9,NAVER,694000,4000,0.58,228761,92824


In [8]:
url = make_url(1406,1)
df = get_data(url)
print(len(df))
df.tail()

1406


,종목,시세,전일비,등락율,시가총액,거래량
1401,KODEX 미국달러선물인버스,10775,-90,-0.83,54,3738
1402,KODEX 독일MSCI(합성),10715,30,0.28,54,198
1403,TIGER 200 금융,8080,85,1.06,53,109169
1404,TIGER 코스피대형주,11405,-90,-0.78,51,1
1405,KODEX 구리선물(H),6390,-145,-2.22,51,1359


In [40]:
# for문을 쓰지 않고 패키지를 이용해서 데이터 정리
from pandas.io.json import json_normalize
def get_data(url):
    response = requests.get(url)  
    json_info = response.json()   #JSON으로 변환
    companys = json_info['result']['itemList']
    return pd.DataFrame(companys)

In [41]:
url = make_url()
get_data(url)

,aa,aq,cd,cr,cv,mks,mt,nm,nv,pcv,rf
0,808076,16839154,005930,-1.14,-550,3058808,0,삼성전자,47650,48200,5
1,255711,2936604,000660,0.81,700,633362,0,SK하이닉스,87000,86300,2
2,536055,1825018,068270,6.61,18500,374307,0,셀트리온,298500,280000,2
3,91322,2402876,005935,-2.45,-950,341572,0,삼성전자우,37800,38750,5
4,85210,232924,005490,-1.22,-4500,318232,0,POSCO,365000,369500,5
5,98712,732530,005380,-0.74,-1000,295170,0,현대차,134000,135000,5
6,75421,183984,207940,0.00,0,269953,0,삼성바이오로직스,408000,408000,3
7,99414,264303,051910,0.95,3500,263662,0,LG화학,373500,370000,2
8,47783,391362,028260,-0.41,-500,231422,0,삼성물산,122000,122500,5
9,74164,1356696,105560,-1.79,-1000,229961,0,KB금융,55000,56000,5


#### darksky api
- 위도와 경도를 입력하면 날씨 정보를 보내주는 api
- https://api.darksky.net/
- forecastio 패키지를 이용해서 크롤링

In [42]:
FORECAST_TOKEN = "52ac1d000782c7a453414e97ceae0409"  # 홈페이지에 로그인 하면 제공

In [43]:
def forecast(lat, lng):
    url = "https://api.darksky.net/forecast/{}/{},{}".format(FORECAST_TOKEN, lat, lng)
    response = requests.get(url)
    json_info = response.json()
    json_info
    return json_info["timezone"], json_info["hourly"]['summary']

In [44]:
lat = 37.512
lng = 126.954
forecast(lat, lng)

('Asia/Seoul', 'Mostly cloudy throughout the day.')

In [45]:
# API가 복잡해지면 패키지를 이용하는 게 더 편함
def forecast2(lat, lng):
    forecast = forecastio.load_forecast(FORECAST_TOKEN, lat, lng)
    byHourly = forecast.hourly()
    return byHourly.summary

In [46]:
lat = 37.512
lng = 126.954
forecast2(lat, lng)

'Mostly cloudy throughout the day.'

#### BS4
- 네이버 키워드 랭킹 데이터 크롤링
- 다음 키워드 랭킹 데이터 크롤링

개발자 페이지에서 <Elements 탭>에서 실시간 검색어 위치 찾아서 html코드 분석

In [47]:
# dom.select : 여러개의 html 엘리먼트를 셀렉팅 할 때 사용, 결과로 리스트 데이터를 리턴
# dom.select_one : 하나의 html 엘리먼트를 셀렉팅 할 때 사용, 결과로 문자열 데이터를 리턴
def naver():
    df = pd.DataFrame(columns=["rank", "keyword"])
    
    response = requests.get("https://www.naver.com/")       #url을 문자열로 가져옴
    dom = BeautifulSoup(response.content, 'html.parser')    # charset이 'utf-8'이 아닌경우 from_endocing=에서 변환, html 이 dom 객체로 나옴
    keywords = dom.select(".ah_roll_area.PM_CL_realtimeKeyword_rolling\
                                                      > .ah_l > .ah_item") # class가 두개 적용된 엘리먼트 일경우 .<클래스명>.<클래스명>
                                                        # print(len(keywards))로 해당 class의 개수와 가져오려는 데이터 수가 같은지 확인
    for keyword in keywords:
        df.loc[len(df)] = {
            "rank" : keyword.select_one(".ah_r").text,
            "keyword" : keyword.select_one(".ah_k").text
        }                             
    return df
naver()

,rank,keyword
0,1,이집트 우루과이
1,2,정진철
2,3,쌈디
3,4,블랙핑크
4,5,블랙핑크 뚜두뚜두
5,6,러시아 사우디 하이라이트
6,7,포항 약국
7,8,살라
8,9,심영은
9,10,모로코 이란


In [49]:
import time
ls = []
for idx in range(3):
    ls.append(naver())
    time.sleep(60*1)              # 60초마다 세번 데이터를 수집하여 ls 리스트에 추가

In [55]:
print(ls)

[   rank        keyword
0     1       이집트 우루과이
1     2            정진철
2     3             쌈디
3     4           블랙핑크
4     5      블랙핑크 뚜두뚜두
5     6  러시아 사우디 하이라이트
6     7          포항 약국
7     8             살라
8     9            심영은
9    10         모로코 이란
10   11         프로듀스48
11   12            강혁민
12   13          페미니스트
13   14       포르투갈 스페인
14   15          나혼자산다
15   16            김무성
16   17        페미니스트 뜻
17   18            ksl
18   19            흑산도
19   20         월드컵 중계,    rank        keyword
0     1       이집트 우루과이
1     2            정진철
2     3           블랙핑크
3     4             쌈디
4     5      블랙핑크 뚜두뚜두
5     6  러시아 사우디 하이라이트
6     7          포항 약국
7     8             살라
8     9         모로코 이란
9    10         프로듀스48
10   11            강혁민
11   12          페미니스트
12   13       포르투갈 스페인
13   14            김무성
14   15          나혼자산다
15   16            흑산도
16   17         월드컵 중계
17   18        페미니스트 뜻
18   19            ksl
19   20         흑산도 여행,    rank        keyword
0     1 

In [56]:
def daum():
    df = pd.DataFrame()
    response = requests.get("https://www.daum.net/")
    dom = BeautifulSoup(response.content, 'html.parser') 
    keywords = dom.select(".list_hotissue.issue_row.list_mini > li") 
    rank = [keyword.select_one(".ir_wa").text for keyword in keywords]
    keywords = [keyword.select_one(".link_issue").text for keyword in keywords]
    df["rank"] = rank
    df["keyword"] = keywords
    return df

daum()    

,rank,keyword
0,1위,블랙핑크
1,2위,안희정
2,3위,자유한국당
3,4위,포항 약국
4,5위,심영은
5,6위,이집트 우루과이
6,7위,민갑룡
7,8위,살라
8,9위,송철호
9,10위,이찬오


In [57]:
naver_df = naver()
daum_df = daum()

In [58]:
# Naver키워드와 Daum키워드 비교해서 공통 단어 추출
# 방법 1) for comprehension
result = [keyword for keyword in naver_df["keyword"] if daum_df["keyword"].str.contains(keyword).any()]
result

['이집트 우루과이', '블랙핑크', '살라', '심영은', '안희정']

In [59]:
# 방법 2) set과 교집합 이용
set(naver_df["keyword"]) & set(daum_df["keyword"])

{'블랙핑크', '살라', '심영은', '안희정', '이집트 우루과이'}

#### file down load : requests
- file의 url을 받아서 해당 url의 file을 다운로드 하는 방법
- url이 길 경우 https://bitly.com/ 에서 단축url을 생성

In [60]:
def download(title, download_link):
    response = requests.get(download_link, stream=True)  #파일을 다운받을 것이므로 stream = True
    file_size = 0
    with open(title, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024): # byte 단위
        #chunk(잘라서 저장)를 사용 이유:파일을 전송하다가 깨지는 부분이 발생하면 재전송 필요하므로 잘게 잘라서 보내면 그 부분만 재전송 가능
            if chunk:
                file_size += 1024
                f.write(chunk)
    return file_size

In [61]:
download_link = "https://bit.ly/2yds515"
title = "video.mp4"
file_size = download(title, download_link)

In [62]:
str(round(file_size/1024/1024, 2)) + "Mb"

'2.99Mb'